In [17]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 16: Flawed Frequency Transmission ---</h2><p>You're 3/4ths of the way through the <a href="https://en.wikipedia.org/wiki/Gas_giant">gas giants</a>. Not only do roundtrip signals to Earth take <span title="In minutes, that's as many as thirty tens!">five hours</span>, but the signal quality is quite bad as well.  You can clean up the signal with the Flawed Frequency Transmission algorithm, or <em>FFT</em>.</p>
<p>As input, FFT takes a list of numbers.  In the signal you received (your puzzle input), each number is a single digit: data like <code>15243</code> represents the sequence <code>1</code>, <code>5</code>, <code>2</code>, <code>4</code>, <code>3</code>.</p>
<p>FFT operates in repeated <em>phases</em>.  In each phase, a new list is constructed with the same length as the input list.  This new list is also used as the input for the next phase.</p>
<p>Each element in the new list is built by multiplying every value in the input list by a value in a repeating <em>pattern</em> and then adding up the results. So, if the input list were <code>9, 8, 7, 6, 5</code> and the pattern for a given element were <code>1, 2, 3</code>, the result would be <code>9*1 + 8*2 + 7*3 + 6*1 + 5*2</code> (with each input element on the left and each value in the repeating pattern on the right of each multiplication). Then, only the ones digit is kept: <code>38</code> becomes <code>8</code>, <code>-17</code> becomes <code>7</code>, and so on.</p>
<p>While each element in the output array uses all of the same input array elements, the actual repeating pattern to use depends on <em>which output element</em> is being calculated. The base pattern is <code>0, 1, 0, -1</code>.  Then, repeat each value in the pattern a number of times equal to the <em>position in the output list</em> being considered. Repeat once for the first element, twice for the second element, three times for the third element, and so on.  So, if the third element of the output list is being calculated, repeating the values would produce: <code>0, 0, 0, 1, 1, 1, 0, 0, 0, -1, -1, -1</code>.</p>
<p>When applying the pattern, skip the very first value exactly once. (In other words, offset the whole pattern left by one.) So, for the second element of the output list, the actual pattern used would be: <code>0, 1, 1, 0, 0, -1, -1, 0, 0, 1, 1, 0, 0, -1, -1, ...</code>.</p>
<p>After using this process to calculate each element of the output list, the phase is complete, and the output list of this phase is used as the new input list for the next phase, if any.</p>
<p>Given the input signal <code>12345678</code>, below are four phases of FFT. Within each phase, each output digit is calculated on a single line with the result at the far right; each multiplication operation shows the input digit on the left and the pattern value on the right:</p>
<pre><code>Input signal: 12345678

```
1 * 1 + 2 * 0 + 3 * -1 + 4 * 0 + 5 * 1 + 6 *  0 + 7 * -1 + 8 * 0 = 4
1 * 0 + 2 * 1 + 3 *  1 + 4 * 0 + 5 * 0 + 6 * -1 + 7 * -1 + 8 * 0 = 8
1 * 0 + 2 * 0 + 3 *  1 + 4 * 1 + 5 * 1 + 6 *  0 + 7 *  0 + 8 * 0 = 2
1 * 0 + 2 * 0 + 3 *  0 + 4 * 1 + 5 * 1 + 6 *  1 + 7 *  1 + 8 * 0 = 2
1 * 0 + 2 * 0 + 3 *  0 + 4 * 0 + 5 * 1 + 6 *  1 + 7 *  1 + 8 * 1 = 6
1 * 0 + 2 * 0 + 3 *  0 + 4 * 0 + 5 * 0 + 6 *  1 + 7 *  1 + 8 * 1 = 1
1 * 0 + 2 * 0 + 3 *  0 + 4 * 0 + 5 * 0 + 6 *  0 + 7 *  1 + 8 * 1 = 5
1 * 0 + 2 * 0 + 3 *  0 + 4 * 0 + 5 * 0 + 6 *  0 + 7 *  0 + 8 * 1 = 8

After 1 phase: 48226158

4 * 1 + 8 * 0 + 2 * -1 + 2 * 0 + 6 * 1 + 1 *  0 + 5 * -1 + 8 * 0 = 3
4 * 0 + 8 * 1 + 2 *  1 + 2 * 0 + 6 * 0 + 1 * -1 + 5 * -1 + 8 * 0 = 4
4 * 0 + 8 * 0 + 2 *  1 + 2 * 1 + 6 * 1 + 1 *  0 + 5 *  0 + 8 * 0 = 0
4 * 0 + 8 * 0 + 2 *  0 + 2 * 1 + 6 * 1 + 1 *  1 + 5 *  1 + 8 * 0 = 4
4 * 0 + 8 * 0 + 2 *  0 + 2 * 0 + 6 * 1 + 1 *  1 + 5 *  1 + 8 * 1 = 0
4 * 0 + 8 * 0 + 2 *  0 + 2 * 0 + 6 * 0 + 1 *  1 + 5 *  1 + 8 * 1 = 4
4 * 0 + 8 * 0 + 2 *  0 + 2 * 0 + 6 * 0 + 1 *  0 + 5 *  1 + 8 * 1 = 3
4 * 0 + 8 * 0 + 2 *  0 + 2 * 0 + 6 * 0 + 1 *  0 + 5 *  0 + 8 * 1 = 8

After 2 phases: 34040438

3 * 1 + 4 * 0 + 0 * -1 + 4 * 0 + 0 * 1 + 4 *  0 + 3 * -1 + 8 * 0 = 0
3 * 0 + 4 * 1 + 0 *  1 + 4 * 0 + 0 * 0 + 4 * -1 + 3 * -1 + 8 * 0 = 3
3 * 0 + 4 * 0 + 0 *  1 + 4 * 1 + 0 * 1 + 4 *  0 + 3 *  0 + 8 * 0 = 4
3 * 0 + 4 * 0 + 0 *  0 + 4 * 1 + 0 * 1 + 4 *  1 + 3 *  1 + 8 * 0 = 1
3 * 0 + 4 * 0 + 0 *  0 + 4 * 0 + 0 * 1 + 4 *  1 + 3 *  1 + 8 * 1 = 5
3 * 0 + 4 * 0 + 0 *  0 + 4 * 0 + 0 * 0 + 4 *  1 + 3 *  1 + 8 * 1 = 5
3 * 0 + 4 * 0 + 0 *  0 + 4 * 0 + 0 * 0 + 4 *  0 + 3 *  1 + 8 * 1 = 1
3 * 0 + 4 * 0 + 0 *  0 + 4 * 0 + 0 * 0 + 4 *  0 + 3 *  0 + 8 * 1 = 8

After 3 phases: 03415518

0 * 1 + 3 * 0 + 4 * -1 + 1 * 0 + 5 * 1 + 5 *  0 + 1 * -1 + 8 * 0 = 0
0 * 0 + 3 * 1 + 4 *  1 + 1 * 0 + 5 * 0 + 5 * -1 + 1 * -1 + 8 * 0 = 1
0 * 0 + 3 * 0 + 4 *  1 + 1 * 1 + 5 * 1 + 5 *  0 + 1 *  0 + 8 * 0 = 0
0 * 0 + 3 * 0 + 4 *  0 + 1 * 1 + 5 * 1 + 5 *  1 + 1 *  1 + 8 * 0 = 2
0 * 0 + 3 * 0 + 4 *  0 + 1 * 0 + 5 * 1 + 5 *  1 + 1 *  1 + 8 * 1 = 9
0 * 0 + 3 * 0 + 4 *  0 + 1 * 0 + 5 * 0 + 5 *  1 + 1 *  1 + 8 * 1 = 4
0 * 0 + 3 * 0 + 4 *  0 + 1 * 0 + 5 * 0 + 5 *  0 + 1 *  1 + 8 * 1 = 9
0 * 0 + 3 * 0 + 4 *  0 + 1 * 0 + 5 * 0 + 5 *  0 + 1 *  0 + 8 * 1 = 8

After 4 phases: 01029498
```

</code></pre>

<p>Here are the first eight digits of the final output list after 100 phases for some larger inputs:</p>
<ul>
<li><code>80871224585914546619083218645595</code> becomes <code>24176176</code>.</li>
<li><code>19617804207202209144916044189917</code> becomes <code>73745418</code>.</li>
<li><code>69317163492948606335995924319873</code> becomes <code>52432133</code>.</li>
</ul>
<p>After <em>100</em> phases of FFT, <em>what are the first eight digits in the final output list?</em></p>
</article>


In [18]:
tests = [
    {
        "name": "Example 1-1",
        "signal": "12345678",
        "phases": 1,
        "expected": "48226158",
    },
    {
        "name": "Example 1-2",
        "signal": "12345678",
        "phases": 2,
        "expected": "34040438",
    },
    {
        "name": "Example 1-3",
        "signal": "12345678",
        "phases": 3,
        "expected": "03415518",
    },
    {
        "name": "Example 1-4",
        "signal": "12345678",
        "phases": 4,
        "expected": "01029498",
    },
    {
        "name": "Example 2",
        "signal": "80871224585914546619083218645595",
        "phases": 100,
        "expected": "24176176",
    },
    {
        "name": "Example 3",
        "signal": "19617804207202209144916044189917",
        "phases": 100,
        "expected": "73745418",
    },
    {
        "name": "Example 4",
        "signal": "69317163492948606335995924319873",
        "phases": 100,
        "expected": "52432133",
    },
]


def fft(signal: str, phases) -> str:
    n = len(signal)
    seq = [int(i) for i in signal]
    pattern = [0, 1, 0, -1]
    vs = [0] * n
    for _ in range(phases):
        for m in range(1, n + 1):
            v = 0
            for i in range(m, n + 1):
                v += pattern[i // m % 4] * seq[i - 1]
            vs[m - 1] = abs(v) % 10
        seq = vs[:]
    return "".join(str(i) for i in seq[:8])


run_tests_params(fft, tests)


Test Example 1-1 passed, for fft.
Test Example 1-2 passed, for fft.
Test Example 1-3 passed, for fft.
Test Example 1-4 passed, for fft.
Test Example 2 passed, for fft.
Test Example 3 passed, for fft.
Test Example 4 passed, for fft.
Success


In [19]:
with open("../input/day16.txt") as f:
    puzzle = f.read().strip()

print(f"Part I: {fft(puzzle, 100)}")

Part I: 59522422


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>59522422</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now that your FFT is working, you can decode the <em>real signal</em>.</p>
<p>The real signal is your puzzle input <em>repeated 10000 times</em>. Treat this new signal as a single input list. Patterns are still calculated as before, and 100 phases of FFT are still applied.</p>
<p>The <em>first seven digits</em> of your initial input signal also represent the <em>message offset</em>. The message offset is the location of the eight-digit message in the final output list. Specifically, the message offset indicates <em>the number of digits to skip</em> before reading the eight-digit message. For example, if the first seven digits of your initial input signal were <code>1234567</code>, the eight-digit message would be the eight digits after skipping 1,234,567 digits of the final output list. Or, if the message offset were <code>7</code> and your final output list were <code>98765432109876543210</code>, the eight-digit message would be <code>21098765</code>. (Of course, your real message offset will be a seven-digit number, not a one-digit number like <code>7</code>.)</p>
<p>Here is the eight-digit message in the final output list after 100 phases. The message offset given in each input has been highlighted. (Note that the inputs given below are repeated 10000 times to find the actual starting input lists.)</p>
<ul>
<li><code><em>0303673</em>2577212944063491565474664</code> becomes <code>84462026</code>.</li>
<li><code><em>0293510</em>9699940807407585447034323</code> becomes <code>78725270</code>.</li>
<li><code><em>0308177</em>0884921959731165446850517</code> becomes <code>53553731</code>.</li>
</ul>
<p>After repeating your input signal 10000 times and running 100 phases of FFT, <em>what is the eight-digit message embedded in the final output list?</em></p>
</article>

</main>


In [20]:
tests = [
    {
        "name": "Example 1",
        "signal": "03036732577212944063491565474664",
        "expected": "84462026",
    },
    {
        "name": "Example 2",
        "signal": "02935109699940807407585447034323",
        "expected": "78725270",
    },
    {
        "name": "Example 3",
        "signal": "03081770884921959731165446850517",
        "expected": "53553731",
    },
]


def fft_II(signal: str) -> str:
    offset = int(signal[:7])
    seq = ([int(i) for i in signal] * 10_000)[offset:]
    n = len(seq)

    for _ in range(100):
        sum = 0
        for i in range(n - 1, -1, -1):
            sum += seq[i]
            seq[i] = sum % 10

    return "".join(str(i) for i in seq[:8])


run_tests_params(fft_II, tests)


Test Example 1 passed, for fft_II.
Test Example 2 passed, for fft_II.
Test Example 3 passed, for fft_II.
Success


In [21]:
with open("../input/day16.txt") as f:
    puzzle = f.read().strip()

print(f"Part II: {fft_II(puzzle)}")

Part II: 18650834


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>18650834</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


In [23]:
from math import prod
from util import *


def parse_input(input):
    return [int(x) for x in input.strip()]


def to_str(digits):
    return "".join([str(x) for x in digits])


base_pattern = [0, 1, 0, -1]


def get_pattern(num_count, i):
    pattern = []
    pi = 0
    while len(pattern) < num_count + 1:
        n = base_pattern[pi]
        pattern += [n] * (i + 1)
        pi = (pi + 1) % len(base_pattern)
    return pattern[1 : num_count + 1]


def apply_fft(nums, rounds):
    nums = nums[:]
    c = len(nums)
    for _ in range(rounds):
        for i in range(c):
            s = 0
            sign = 1
            for j in range(i, c, 2 * (i + 1)):
                s += sign * sum(nums[j : j + i + 1])
                sign = -sign
            nums[i] = abs(s) % 10
    return nums


# We want to compute A^count where
#
#     [ 1 1 ... 1 1 ]
#     [ 0 1 ... 1 1 ]
# A = [ ...     ... ]
#     [ 0 0 ... 1 1 ]
#     [ 0 0 ... 0 1 ]
#
# i.e. where A has 1s on and above the diagonal.
#
# If you compute A^2, you notice that it looks like
#
#       [ 1 2 ... n-1   n ]
#       [ 0 1 ... n-2 n-1 ]
# A^2 = [ ... ... ... ... ]
#       [ 0 0 ...   1   2 ]
#       [ 0 0 ...   0   1 ]
#
# i.e., each row is the row above shifted 1 to the right, and the entries
# in the first row are the sum of the columns of A. Similarly, the rows of
# A^3 are shifts of the first row, and the first row are the sums of the
# columns of A^2, namely the sums of the first n integers. Therefore, the
# first row of A^3 are the triangular numbers:
#
#       [ 1 3 6 10  ... ]
#       [ 0 1 3  6  ... ]
# A^3 = [       ...     ]
#       [ 0 0 0 ... 1 3 ]
#       [ 0 0 0 ... 0 1 ]
#
# The formula for the nth triangular number is:
#
#   T_{n,3} = B(n+1, 2) = n*(n+1)/2,
#
# where B(n, k) = n!/(k!*(n-k)!) are the binomial coefficients.
#
# Similarly, the first row of A^4 are the tetrahedral numbers, which are
# the sum of the first n triangular numbers. The formula for the nth
# tetrahedral number is:
#
#   T_{n,4} = B(n+2, 3) = n*(n+1)*(n+2)/3.
#
# We can then guess that the first row of A^k follows the formula:
#
#   T_{n,k} = B(n+k-2, k-1).
#
# This is in fact true, and we can show this by showing:
#
#  B(n+k-1, k) = ∑_{m=1}^n B(m+k-2, k-1),
#
# which follows from https://en.wikipedia.org/wiki/Hockey-stick_identity .


def binom(n, k):
    return prod([n - i for i in range(k)]) // prod(range(1, k + 1))


# Straightforward (but slow) implementation.


def T_k_slow(max_n, k, modulus):
    return [binom(n + k - 2, k - 1) % modulus for n in range(1, max_n + 1)]


# Compute the binomial coefficients using a running product.


def T_k_fast(max_n, k, modulus):
    out = [0] * max_n
    x = 1
    out[0] = 1
    for n in range(2, max_n + 1):
        x *= n + k - 2
        x //= n - 1
        out[n - 1] = x % modulus
    return out


# Use https://en.wikipedia.org/wiki/Lucas%27s_theorem
# to compute binom(n, k) % p efficiently for prime p.


def binom_mod_p(n, k, p):
    prod = 1
    while True:
        n, n_i = divmod(n, p)
        k, k_i = divmod(k, p)
        prod = (prod * binom(n_i, k_i)) % p
        if n == 0 and k == 0:
            break
    return prod


# Finally, use https://en.wikipedia.org/wiki/Chinese_remainder_theorem to
# compute binom(n, k) % 10 in terms of binom(n, k) % 2 and binom(n, k) % 5.
#
# Since (-2)*2 + 1*5 = 1, the CRT says that if n = a_1 mod 2 and n = a_2 mod 5,
# then n = (5*a_1 - 4*a_2) mod 10.
#
# This is actually slower than T_k_fast above for small inputs,
# though. It's only used for
# https://www.reddit.com/r/adventofcode/comments/ebb8w6/2019_day_16_part_three_a_fanfiction_by_askalski/ .


def binom_mod_10(n, k):
    # Compute binom(n, k) % 2 with bitwise operators.
    a1 = int(not (~n & k))
    a2 = binom_mod_p(n, k, 5)
    return (5 * a1 - 4 * a2) % 10


def T_k_asympt_fastest(max_n, k, m):
    if m != 10:
        raise Exception("unexpected m={m}")
    return [binom_mod_10(n + k - 2, k - 1) for n in range(1, max_n + 1)]


def apply_fft_second_half(nums_in, rounds, c, T_k):
    modulus = 10
    coeffs = T_k(len(nums_in), rounds, modulus)
    return [
        sum([(x * y) % modulus for x, y in zip(nums_in[i:], coeffs)]) % modulus
        for i in range(c)
    ]


def extract_message(input, rounds, T_k):
    nums_in = parse_input(input)
    offset = int(input[:7])
    extended_nums_in = nums_in * 10000
    output = apply_fft_second_half(extended_nums_in[offset:], rounds, 8, T_k)
    return to_str(output)


def compute_day16(input):
    nums_in = parse_input(input)
    output_part1 = apply_fft(nums_in, 100)
    part1 = to_str(output_part1[:8])

    part2 = extract_message(input, 100, T_k_fast)
    return part1, part2


with open("../input/day16.txt", "r") as input_file:
    input = input_file.read()
    p1, p2 = compute_day16(input)
    print(f"part 1: {p1}, part 2: {p2}")

with open("../input/day16.txt", "r") as input_file:
    input = input_file.read()
    p3 = extract_message(input, 287029238942, T_k_asympt_fastest)
    print(f"part 3: {p3}")

part 1: 59522422, part 2: 18650834
part 3: 69860303
